# Context-sensitive Spelling Correction

In [1]:
import re
from tqdm import tqdm
import string
import nltk
from nltk.corpus import brown, reuters, webtext
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.util import bigrams, trigrams
from functools import lru_cache
import math
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

## Norvig's Solution
Firstly, I decided to implement Norvig's solution as a baseline for future comparison. To ensure a fair comparison, I will use his original dataset rather than an expanded version. This allows me to measure my future improvements objectively when incorporating additional techniques later.

In [2]:
def words(text): return re.findall(r'\w+', text.lower())


WORDS = Counter(words(open('data/big.txt').read()))


def P(word, N=sum(WORDS.values())):
    return WORDS[word] / N


def correction(word):
    return max(candidates(word), key=P)


def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


def known(words):
    return set(w for w in words if w in WORDS)


def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [3]:
def norvig_sentence_correction(sentence):
    if not sentence.strip():
        return sentence

    words = word_tokenize(sentence)
    corrected_words = [correction(word) for word in words]

    return " ".join(corrected_words)

### Evaluation
To evaluate the spelling correction model, I will use the following metric:
- Word F1-score
- Character Accuracy
- Word Error Rate (WER)

In [4]:
from sklearn.metrics import f1_score
import numpy as np
import Levenshtein


def word_f1(true_words, corrected_words):
    min_length = min(len(true_words), len(corrected_words))
    true_words = true_words[:min_length]
    corrected_words = corrected_words[:min_length]

    return f1_score(true_words, corrected_words, average='micro')


def word_error_rate(true_sentence, corrected_sentence):
    true_words = word_tokenize(true_sentence)
    return Levenshtein.distance(true_sentence, corrected_sentence) / len(true_words)


def evaluate_spelling_correction(true_sentences, corrected_sentences):
    word_f1_scores = []
    char_accuracies = []
    wer_scores = []

    for true_sentence, corrected_sentence in zip(true_sentences, corrected_sentences):
        true_words = word_tokenize(true_sentence)
        corrected_words = word_tokenize(corrected_sentence)

        word_f1_scores.append(word_f1(true_words, corrected_words))

        # Ensure correct handling when sentences differ in length
        char_acc = np.mean([1 if c1 == c2 else 0 for c1, c2 in zip(true_sentence, corrected_sentence)]) \
            if len(true_sentence) == len(corrected_sentence) else 0
        char_accuracies.append(char_acc)

        wer_scores.append(word_error_rate(true_sentence, corrected_sentence))

    print(f"Spelling Correction Evaluation Results:")
    print(f"Word F1-score: {np.mean(word_f1_scores):.2f} (Higher is better)")
    print(f"Character Accuracy: {np.mean(char_accuracies):.2f} (Higher is better)")
    print(f"Word Error Rate (WER): {np.mean(wer_scores):.2f} (Lower is better)")

For the evaluation part I find a dataset on Hugging Face that include only two types of sentences: 
- *Target*: sentences without spelling errors.
- *Source*: sentences with spelling mistakes

In [5]:
from datasets import load_dataset

ds = load_dataset("vishnun/SpellGram")['train']
ds

Dataset({
    features: ['source', 'target'],
    num_rows: 40000
})

Test dataset is large enough, so I will use only part of it

In [6]:
true_sentences = ds['target'][1000:7000]
corrected_sentences = ds['source'][1000:7000]

In [7]:
improved_sentences = []

for sentence in tqdm(corrected_sentences):
    improved_sentences.append(norvig_sentence_correction(sentence))

evaluate_spelling_correction(true_sentences, improved_sentences)

100%|██████████| 6000/6000 [03:51<00:00, 25.96it/s]


Spelling Correction Evaluation Results:
Word F1-score: 0.85 (Higher is better)
Character Accuracy: 0.63 (Higher is better)
Word Error Rate (WER): 0.21 (Lower is better)


## My implementation
### Data preprocessing
As the first improvement, I will use a dataset from the NLTK library. This dataset contains a large vocabulary, which will help improve spelling correction by providing a more comprehensive reference for word probabilities and possible corrections. A larger corpus increases the chances of correctly identifying and ranking candidate words, leading to better overall accuracy.

In [8]:
nltk.download("brown")
nltk.download("reuters")
nltk.download("webtext")
nltk.download("punkt")

texts = brown.sents() + reuters.sents() + webtext.sents()

tokens = []
for sent in texts:
    clean_sent = [word.lower() for word in sent if word.isalnum()]
    if clean_sent:
        tokens.extend(["<s>"] + clean_sent + ["</s>"])

unigram_freq = Counter(tokens)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
bigram_list = [bg for bg in bigrams(tokens) if "<s>" not in bg and "</s>" not in bg]
trigram_list = [tg for tg in trigrams(tokens) if "<s>" not in tg and "</s>" not in tg]

bigram_freq = Counter(bigram_list)
trigram_freq = Counter(trigram_list)

In [10]:
len(unigram_freq), len(bigram_freq), len(trigram_freq)

(63598, 803180, 1679220)

### Keyboard mapping
 I implemented keyboard mapping, which takes into account common typing errors caused by key proximity. This approach assigns higher probabilities to mistakes that are likely due to adjacent keys on a standard QWERTY keyboard. For example, mistyping "g" instead of "f" is more likely than replacing it with "z."

In [11]:
keyboard = {
    'a': 'qwsz',
    'b': 'vghn',
    'c': 'xdfv',
    'd': 'erfcxs',
    'e': 'rdsw',
    'f': 'rtgvcd',
    'g': 'tyhbvf',
    'h': 'yujnbg',
    'i': 'ujko',
    'j': 'uikmnh',
    'k': 'iolmj',
    'l': 'opk',
    'm': 'njk',
    'n': 'bhjm',
    'o': 'iklp',
    'p': 'ol',
    'q': 'wa',
    'r': 'etdf',
    's': 'wedxza',
    't': 'rygf',
    'u': 'yihj',
    'v': 'cfgb',
    'w': 'qesa',
    'x': 'zsdc',
    'y': 'tghu',
    'z': 'asx'
}

In [12]:
def keyboard_penalty(correct, wrong):
    if correct == wrong:
        return 1
    elif wrong in keyboard.get(correct, ""):
        return 0.95
    else:
        return 0.7

### Correction algorithm

In [13]:
import heapq


def generate_keyboard_edits(word):
    edits = set()
    word = word.lower()

    for i, char in enumerate(word):
        if char in keyboard:
            for neighbor in keyboard[char]:
                typo = word[:i] + neighbor + word[i + 1:]
                edits.add((typo, 0.95))

    return edits


def generate_candidates(word):
    letters = string.ascii_lowercase
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    deletes = {(L + R[1:], 0.2) for L, R in splits if R}
    transposes = {(L + R[1] + R[0] + R[2:], 0.5) for L, R in splits if len(R) > 1}
    inserts = {(L + c + R, 0.3) for L, R in splits for c in letters}

    replaces = {
        (L + c + R[1:], keyboard_penalty(R[0], c))
        for L, R in splits if R for c in letters
    }

    keyboard_replaces = generate_keyboard_edits(word)

    return deletes | transposes | replaces | inserts | keyboard_replaces


def generate_edit2_candidates(word, k=10):
    edit1 = generate_candidates(word)
    edit2 = {candidate for candidate, _ in edit1 if candidate not in unigram_freq}

    return set(heapq.nlargest(k, {cand: unigram_freq.get(cand, 0) for cand in edit2}.items(), key=lambda x: x[1]))



In [14]:
@lru_cache(maxsize=100000)
def unigramProbability(word):
    total = sum(unigram_freq.values())
    return unigram_freq.get(word, 0) / total if total > 0 else 0

In [15]:
@lru_cache(maxsize=100000)
def bigramProbability(word1, word2):
    if not word1:
        return unigramProbability(word2)

    return bigram_freq.get((word1, word2), 0) / unigram_freq.get(word1, 1)

In [16]:
@lru_cache(maxsize=100000)
def trigramProbability(word1, word2, word3):
    if not word1:
        return bigramProbability(word2, word3)
    if not word2:
        return unigramProbability(word3)
    return trigram_freq.get((word1, word2, word3), 0) / bigram_freq.get((word1, word2), 1)

### Spelling

In [17]:
def correct_spelling(word, previous_word, previous_prev_word, trigram=False):
    if word in unigram_freq:
        return word

    candidates = generate_candidates(word)
    known_candidates = {(w, _) for w, _ in candidates if w in unigram_freq}

    if not known_candidates:
        edit2_candidates = generate_edit2_candidates(word)
        known_candidates = {(w, _) for w, _ in edit2_candidates if w in unigram_freq}

    if not known_candidates:
        return word

    if trigram:

        best_candidate = max(known_candidates,
                             key=lambda x: math.log(unigramProbability(x[0]) + 1e-10) +
                                           2 * math.log(
                                 bigramProbability(previous_word, x[0]) + 1e-10) +
                                           3 * math.log(trigramProbability(previous_prev_word, previous_word,
                                                                           x[
                                                                               0]) + 1e-10) +
                                           math.log(x[1] + 1e-10) +
                                           math.log(unigram_freq.get(x[0], 1e-10)))

    else:
        best_candidate = max(known_candidates,
                             key=lambda x: math.log(unigramProbability(x[0]) + 1e-10) +
                                           2 * math.log(
                                 bigramProbability(previous_word, x[0]) + 1e-10) +
                                           math.log(x[1] + 1e-10) +
                                           math.log(unigram_freq.get(x[0], 1e-10)))

    return best_candidate[0]


In [18]:
def correct_sentence(sentence, trigram=False):
    if not sentence.strip():
        return sentence

    words = word_tokenize(sentence)
    corrected_words = []
    prev_word, prev_prev_word = "", ""

    for word in words:
        corrected_word = correct_spelling(word, prev_word, prev_prev_word, trigram=trigram)
        corrected_words.append(corrected_word)
        prev_prev_word, prev_word = prev_word, corrected_word

    return " ".join(corrected_words)

In [19]:
correct_sentence("i study at the universoty")

'i study at the university'

### Evaluation

In [20]:
improved_sentences = []

for sentence in tqdm(corrected_sentences):
    improved_sentences.append(correct_sentence(sentence))

evaluate_spelling_correction(true_sentences, improved_sentences)

100%|██████████| 6000/6000 [00:05<00:00, 1035.66it/s]


Spelling Correction Evaluation Results:
Word F1-score: 0.86 (Higher is better)
Character Accuracy: 0.82 (Higher is better)
Word Error Rate (WER): 0.19 (Lower is better)


In [21]:
improved_sentences = []

for sentence in tqdm(corrected_sentences):
    improved_sentences.append(correct_sentence(sentence, trigram=True))

evaluate_spelling_correction(true_sentences, improved_sentences)

100%|██████████| 6000/6000 [00:02<00:00, 2061.14it/s]


Spelling Correction Evaluation Results:
Word F1-score: 0.86 (Higher is better)
Character Accuracy: 0.82 (Higher is better)
Word Error Rate (WER): 0.19 (Lower is better)


# Justification
## N-grams
The primary limitation in Nodvig's solution was the use of a small dataset. To address this, I leveraged much larger datasets from the NLTK library, which provided a more robust base for calculating word probabilities and correcting spelling errors. Additionally, Nodvig's method relied on unigrams, which is less effective for context-dependent words that are related to others in the sentence. 


Additionally, I integrated trigrams into the solution, though I found that they can slow down the algorithm. The impact of trigrams, while noticeable, is not always critical, as the evaluation results with and without trigrams were very similar. Therefore, I left the use of trigrams as an optional feature. If accuracy is the priority, trigrams are valuable, but they can be omitted for speedier performance.

## Keyboard Map and Weights
One important improvement I made was the inclusion of weighted errors. Many common spelling mistakes are caused by the keyboard layout, and so I assigned different weights to various types of errors based on their likelihood of occurrence. These weights were carefully chosen and tested, and here’s how they work:
- Keyboard mistakes (keyboard_replaces): 0.95 (very common errors, such as mistyped adjacent keys)
- Replacements (replaces): 0.8 (frequent but less common than keyboard-related mistakes)
- Transpositions (transposes): 0.5 (less frequent, such as switching adjacent letters)
- Insertions (inserts): 0.3 (rare, when extra characters are typed)
- Deletions (deletes): 0.2 (also rare, where characters are omitted)

This weighted approach allows for better prioritization during the search for potential corrections. It ensures that more frequent errors, such as those arising from adjacent key presses, are given higher priority.

## Improve Speed
Another notable improvement I made was to optimize the edit2 generation process using heapq. Previously, generating all possible candidate edits for the second iteration (edit2) could be computationally expensive. By using a heap, I ensured that only the best candidates were prioritized, and I reduced the impact of low-weighted corrections. 

Also, I implemented caching to speed up the code. All computations are now much faster, as results of frequently repeated operations (such as unigram, bigram, and trigram probabilities) are cached. This avoids redundant calculations and significantly improves performance.

## Best Candidate Function
When selecting the best candidates, I implemented a maximization function, which can be quite large and complex due to the many variables at play. To avoid the issue of probabilities becoming excessively small, I employed logarithms. This ensures that the values stay manageable and helps stabilize the calculation process. Additionally, I adjusted the weights for bigrams and trigrams to give more importance to these models, recognizing their higher contextual value.

## Evaluation Results
The difference between the results is not extremely significant, but my implementation shows much better results for Character Accuracy. I believe this improvement is largely due to the incorporation of bigram and trigram models, which capture more context and help correct mistakes at the character level. Additionally, the speed improvement is quite noticeable, thanks to optimizations like caching and more efficient handling of edit candidates. These changes contribute to both better performance and faster processing.
